#### Autoencoders

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import time
import os
import sys
import torchvision

In [2]:
class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=128
        )
        self.encoder_output_layer = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_hidden_layer = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_output_layer = nn.Linear(
            in_features=128, out_features=kwargs["input_shape"]
        )

    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed

In [3]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=784).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# mean-squared error loss
criterion = nn.MSELoss()

In [5]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(
    root="~/torch_datasets", train=True, transform=transform, download=True
)

test_dataset = torchvision.datasets.MNIST(
    root="~/torch_datasets", train=False, transform=transform, download=True
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=32, shuffle=False, num_workers=4
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 8860948.31it/s] 


Extracting /Users/h6x/torch_datasets/MNIST/raw/train-images-idx3-ubyte.gz to /Users/h6x/torch_datasets/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 346578.28it/s]


Extracting /Users/h6x/torch_datasets/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/h6x/torch_datasets/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 964998.23it/s] 


Extracting /Users/h6x/torch_datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/h6x/torch_datasets/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1532378.44it/s]

Extracting /Users/h6x/torch_datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/h6x/torch_datasets/MNIST/raw



In [6]:
epochs = 30

In [7]:
for epoch in range(epochs):
    loss = 0
    for batch_features, _ in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        batch_features = batch_features.view(-1, 784).to(device)
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

epoch : 1/30, loss = 0.033011
epoch : 2/30, loss = 0.018093
epoch : 3/30, loss = 0.016075
epoch : 4/30, loss = 0.015038
epoch : 5/30, loss = 0.014372
epoch : 6/30, loss = 0.013908
epoch : 7/30, loss = 0.013562
epoch : 8/30, loss = 0.013287
epoch : 9/30, loss = 0.013059
epoch : 10/30, loss = 0.012875
epoch : 11/30, loss = 0.012720
epoch : 12/30, loss = 0.012567
epoch : 13/30, loss = 0.012446
epoch : 14/30, loss = 0.012332
epoch : 15/30, loss = 0.012249
epoch : 16/30, loss = 0.012154
epoch : 17/30, loss = 0.012074
epoch : 18/30, loss = 0.011998
epoch : 19/30, loss = 0.011944
epoch : 20/30, loss = 0.011883
epoch : 21/30, loss = 0.011830
epoch : 22/30, loss = 0.011788
epoch : 23/30, loss = 0.011743
epoch : 24/30, loss = 0.011704
epoch : 25/30, loss = 0.011665
epoch : 26/30, loss = 0.011634
epoch : 27/30, loss = 0.011599
epoch : 28/30, loss = 0.011583
epoch : 29/30, loss = 0.011544
epoch : 30/30, loss = 0.011517


In [10]:
# predict using one image and get the reconstruction loss
for batch_features,_ in test_loader:
    test_examples = batch_features.view(-1, 784).to(device)
    
    output = model(test_examples)
    # print(reconstruction)
    test_loss = criterion(output, test_examples)
    print("Test Loss: {:.6f}".format(test_loss.item()))
    
    break

Test Loss: 0.010092
